In [40]:
import numpy as np
import sympy as sp
import pandas as pd

In [41]:
class LDA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.X_train = None
        self.y_train = None
        self.Sw = None
        self.Sb = None
        self.eigen_values = None
        self.eigen_vectors = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        Sw = np.zeros((self.X_train.shape[1], self.X_train.shape[1]))
        Sb = np.zeros((self.X_train.shape[1], self.X_train.shape[1]))
        Sw = np.zeros((self.X_train.shape[1], self.X_train.shape[1]))
        for i in range(len(self.X_train)):
            x = self.X_train[i].reshape(self.X_train.shape[1], 1)
            m = np.mean(self.X_train[self.y_train == self.y_train[i]])
            Sw += np.dot((x - m), (x - m).T)
        m = np.mean(self.X_train)
        for i in range(len(self.X_train)):
            x = self.X_train[i].reshape(self.X_train.shape[1], 1)
            m_i = np.mean(self.X_train[self.y_train == self.y_train[i]])
            Sb += len(self.X_train[self.y_train == self.y_train[i]]) * np.dot((m_i - m), (m_i - m).T)
        self.Sw = Sw
        self.Sb = Sb
        A = np.dot(np.linalg.inv(Sw), Sb)
        eigen_values, eigen_vectors = np.linalg.eig(A)
        eigen_vectors = eigen_vectors.T
        idx = np.argsort(abs(eigen_values))[::-1]
        eigen_vectors = eigen_vectors[idx]
        eigen_values = eigen_values[idx]
        self.eigen_values = eigen_values[0:self.n_components]
        self.eigen_vectors = eigen_vectors[0:self.n_components]

    def transform(self, X):
        return np.dot(X, self.eigen_vectors.T)

In [42]:
def euclidean_distance(X_train, X_test):
    dist = np.zeros((len(X_test), len(X_train)))
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            dist[i,j] = np.sqrt(np.sum((X_test[i] - X_train[j])**2))
    return dist

In [43]:
def covariance(X):
    mean = np.mean(X,axis=0)
    X = X - mean
    return np.dot(X.T, X)/(X.shape[0]-1)

In [44]:
class KNN:
    def __init__(self, k = 5):
        self.k = k
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        self.distances = euclidean_distance(self.X_train, X_test)
        pred = []
        for dist in self.distances:
            k_nearest_indices = np.argsort(dist)[:self.k]
            k_nearest_labels = self.y_train[k_nearest_indices]
            pred.append(np.unique(k_nearest_labels)[np.argmax(np.unique(k_nearest_labels, return_counts=True)[1])])
        return np.array(pred)

In [45]:
df = pd.read_csv('face.csv')

In [46]:
classes = df['target'].unique()
train = pd.DataFrame()
test = pd.DataFrame()

for i in classes:
    train = pd.concat([train, df[df['target'] == i].iloc[2:]], ignore_index=True)
    test = pd.concat([test, df[df['target'] == i].iloc[:2]], ignore_index=True)

print(test)

           0         1         2         3         4         5         6  \
0   0.309917  0.367769  0.417355  0.442149  0.528926  0.607438  0.657025   
1   0.454545  0.471074  0.512397  0.557851  0.595041  0.640496  0.681818   
2   0.541322  0.586777  0.640496  0.661157  0.685950  0.685950  0.690083   
3   0.644628  0.690083  0.702479  0.702479  0.706612  0.719008  0.727273   
4   0.578512  0.603306  0.632231  0.665289  0.677686  0.710744  0.723140   
..       ...       ...       ...       ...       ...       ...       ...   
75  0.144628  0.219008  0.326446  0.471074  0.570248  0.644628  0.677686   
76  0.252066  0.219008  0.227273  0.272727  0.318182  0.388430  0.458678   
77  0.355372  0.392562  0.446281  0.462810  0.475207  0.491736  0.500000   
78  0.545455  0.611570  0.640496  0.657025  0.636364  0.648760  0.690083   
79  0.334711  0.404959  0.475207  0.537190  0.561983  0.553719  0.586777   

           7         8         9  ...      4087      4088      4089      4090  \
0   0.

In [47]:
X_train = train.drop(['target'], axis=1).values
y_train = train['target'].values
X_test = test.drop(['target'], axis=1).values
y_test = test['target'].values

In [48]:
lda = LDA(39)
lda.fit(X_train, y_train)

In [49]:
X_train = lda.transform(X_train)
X_test = lda.transform(X_test)
print(f"X_train: {len(X_train)}")
print(f"X_test: {len(X_test)}")
print(X_train)

X_train: 320
X_test: 80
[[-3.33867886e-04+0.j         -3.69360494e-05+0.j
  -1.00204240e-02-0.03448946j ... -4.14866716e-03-0.00599499j
  -4.14866716e-03+0.00599499j  6.36530386e-02+0.0124255j ]
 [-3.33867888e-04+0.j          2.81742953e-05+0.j
  -1.05351616e-01-0.19877429j ...  1.84797521e-03-0.03974838j
   1.84797521e-03+0.03974838j  1.65180642e-02+0.08254262j]
 [-3.33867886e-04+0.j         -3.43395995e-05+0.j
  -2.27194320e-02-0.02768734j ...  1.41897907e-01-0.01745156j
   1.41897907e-01+0.01745156j -7.18880417e-02+0.09119231j]
 ...
 [-3.10494538e-04+0.j          1.15070363e-05+0.j
   1.04737282e-02-0.10359843j ...  1.18907382e-01-0.08190134j
   1.18907382e-01+0.08190134j -7.24902514e-02+0.1899742j ]
 [-3.10494538e-04+0.j         -2.00505371e-05+0.j
  -8.93823111e-02-0.23181809j ... -1.57044674e-01-0.05727074j
  -1.57044674e-01+0.05727074j -1.56104293e-02+0.0903929j ]
 [-3.10494539e-04+0.j          1.86561495e-05+0.j
  -1.07453475e-01-0.09157248j ...  4.03278596e-02-0.12283026j
   4

In [50]:
knn = KNN(5)
knn.fit(X_train, y_train)

In [51]:
y_pred = knn.predict(X_test)
compared = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(compared)

/tmp/ipykernel_8164/27565779.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  dist[i,j] = np.sqrt(np.sum((X_test[i] - X_train[j])**2))


    Actual  Predicted
0        0         19
1        0          1
2        1          2
3        1          5
4        2         11
..     ...        ...
75      37          1
76      38         20
77      38         13
78      39          0
79      39          5

[80 rows x 2 columns]


In [52]:
def accuracy(y_pred,y_test):
    return np.sum(y_pred == y_test)/len(y_test)

print("Accuracy: ", accuracy(y_pred, y_test))
print("accuracy percentage: ", accuracy(y_pred, y_test)*100, "%")

Accuracy:  0.05
accuracy percentage:  5.0 %
